## mapping

default json type
true or false -> boolean
123 -> long
12,3 -> double
2014-50-06 -> date
foo bar -> text

text: should be searched in entire context search. so text type should be splited
keyword:not needs to split words 

In [ ]:
GET trials/_mapping

In [ ]:
{
  "trials" : {
    "mappings" : {
      "properties" : {
        "biomarkers" : {
          "properties" : {
            "evs_id" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            },
            "name" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            }
          }
        },
        "create_date" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "id" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "ids" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "is_lead_disease" : {
          "type" : "boolean"
        },
        "name" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "nci_id" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "organizations" : {
          "properties" : {
            "id" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            },
            "name" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            }
          }
        },
        "site" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        }
      }
    }
  }
}


In [ ]:
#add/update  mapping 
PUT trials/_mapping
{
  "properties":{
    "evs_id":{
      "type":"keyword",
      "ignore_above": 512
    }
  }
}

## reindex

1. check current index, and get documents
GET _cat/indices?v=&s=index:desc
For index trials, there are 11 documents

In [ ]:
health status index                           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   trials                          1Ttkj4r5QhGfl64YMF70KQ   1   2         11            4     31.6kb         31.6kb
yellow open   test-index                      BEPc6VLBR4SVXmy6mdJM6w   1   1          1            0      4.8kb          4.8kb
yellow open   orgs-index                      JFgh43E9SKCvd3pPOgr7hw   1   1       5000            0      1.6mb          1.6mb
yellow open   news                            hjQPo_2cSWqrntUpNxiQrw   1   1          0            0       208b           208b
yellow open   new_trials                      iC9_fHSQTK6H8-Ke_w9fww   1   1         11            0     10.2kb         10.2kb
yellow open   ecommerce                       wN-04_2PQ9W3vf8bSyi4Vw   1   1          4            1     12.8kb         12.8kb

2. prepare new mapping properties for new index, and then create a new index
PUT /trials_v2
{
  "settings":{
    "number_of_shards": 2,
    "number_of_replicas": 2
  },
  "mappings" : {
      "properties" : {
        "biomarkers" : {
          "type": "nested",
          "properties" : {
            "evs_id" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword"
                }
              }
            },
            "name" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword"
                }
              }
            }
          }
        },
        "create_date" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "diseases" : {
          "type": "nested",
          "properties" : {
            "evs_id" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword"
                }
              }
            },
            "name" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword"
                }
              }
            }
          }
        },
        "id" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "is_lead_disease" : {
          "type" : "boolean"
        },
        "name" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "nci_id" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "organizations" : {
          "type": "nested",
          "properties" : {
            "id" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword"
                }
              }
            },
            "name" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword"
                }
              }
            }
          }
        },
        "site" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        }
      }
    }
}


In [ ]:
3. transfer data of old index to new index
POST /_reindex
{
  "source": {
    "index": "trials"
  },
  "dest": {
    "index": "trials_v2"
  }
}


In [ ]:
#returns
{
  "took" : 11,
  "timed_out" : false,
  "total" : 11,
  "updated" : 11,
  "created" : 0,
  "deleted" : 0,
  "batches" : 1,
  "version_conflicts" : 0,
  "noops" : 0,
  "retries" : {
    "bulk" : 0,
    "search" : 0
  },
  "throttled_millis" : 0,
  "requests_per_second" : -1.0,
  "throttled_until_millis" : 0,
  "failures" : [ ]
}

In [ ]:
4. check documents of new index
GET trials_v2/_search
{
  "query": {
    "match_all": {}
  }
}

In [ ]:
5. delete old index
DELETE /trials

In [ ]:
6. create alias for new index

In [ ]:
#check alias
GET /_alias
  "trials_v2" : {
    "aliases" : { }
  },

In [ ]:
#create alias
POST /_aliases
{
  "actions": [
    {
      "add": {
        "index": "trials_v2",
        "alias": "trials"
      }
    }
  ]
}

  "trials_v2" : {
    "aliases" : {
      "trials" : { }
    }
  },

## analyzer

In [ ]:
POST /_analyze
{
  "analyzer": "standard",
  "text": ["hello word", "Hi CARY, good morning"]
}

In [ ]:
{
  "tokens" : [
    {
      "token" : "hello",
      "start_offset" : 0,
      "end_offset" : 5,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "word",
      "start_offset" : 6,
      "end_offset" : 10,
      "type" : "<ALPHANUM>",
      "position" : 1
    },
    {
      "token" : "hi",
      "start_offset" : 11,
      "end_offset" : 13,
      "type" : "<ALPHANUM>",
      "position" : 2
    },
    {
      "token" : "cary",
      "start_offset" : 14,
      "end_offset" : 18,
      "type" : "<ALPHANUM>",
      "position" : 3
    },
    {
      "token" : "good",
      "start_offset" : 20,
      "end_offset" : 24,
      "type" : "<ALPHANUM>",
      "position" : 4
    },
    {
      "token" : "morning",
      "start_offset" : 25,
      "end_offset" : 32,
      "type" : "<ALPHANUM>",
      "position" : 5
    }
  ]
}


## settings

In [ ]:
GET trials/_settings

In [ ]:
{
  "trials" : {
    "settings" : {
      "index" : {
        "routing" : {
          "allocation" : {
            "include" : {
              "_tier_preference" : "data_content"
            }
          }
        },
        "number_of_shards" : "1",
        "provided_name" : "trials",
        "creation_date" : "1621776996357",
        "number_of_replicas" : "1",
        "uuid" : "1Ttkj4r5QhGfl64YMF70KQ",
        "version" : {
          "created" : "7100199"
        }
      }
    }
  }
}

In [ ]:
#sholde close index before updating settings
POST /trials/_close
PUT /trials/_settings
{
  "index":{
      "number_of_replicas": "2"
  }
}
POST /trials/_open
GET /trials/_settings

In [ ]:
# POST /trials/_close
{
  "acknowledged" : true,
  "shards_acknowledged" : false,
  "indices" : { }
}

# PUT /trials/_settings
{
  "acknowledged" : true
}

# POST /trials/_open
{
  "acknowledged" : true,
  "shards_acknowledged" : true
}

# GET /trials/_settings
{
  "trials" : {
    "settings" : {
      "index" : {
        "routing" : {
          "allocation" : {
            "include" : {
              "_tier_preference" : "data_content"
            }
          }
        },
        "number_of_shards" : "1",
        "provided_name" : "trials",
        "creation_date" : "1621776996357",
        "number_of_replicas" : "2",
        "uuid" : "1Ttkj4r5QhGfl64YMF70KQ",
        "version" : {
          "created" : "7100199"
        }
      }
    }
  }
}
